In [ ]:
from pynq_dpu import DpuOverlay
overlay = DpuOverlay("attention.bit")

In [ ]:
overlay.load_model("attention-cnn.xmodel")

In [ ]:
import torch
import torchvision
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms,utils
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split,Dataset
from tqdm.notebook import tqdm as tqdm
import torch.utils.data as data
import torchvision.models as models
import seaborn as sns
import torch.quantization as quantization
import warnings
import pickle
import cv2
warnings.filterwarnings('ignore')
sns.set_theme()
import torchvision.transforms as transforms
from torchvision import datasets, transforms
%matplotlib inline
plt.ion() #Turn interactive mode on.
torch.manual_seed(34)
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import time
import tensorflow as tf

In [ ]:
if not os.path.exists('./datasets/visdrone_new/'):
    os.mkdir('./datasets/visdrone_new/')

In [ ]:
train_root_dir = './datasets/visdrone_new/VisDrone2019-DET-train'
val_root_dir = './datasets/visdrone_new/VisDrone2019-DET-val'
test_root_dir = './datasets/visdrone_new/'

In [ ]:
image_list_train = sorted(os.listdir(os.path.join(train_root_dir, 'images')))
label_list_train = sorted(os.listdir(os.path.join(train_root_dir, 'annotations')))

image_list_val = sorted(os.listdir(os.path.join(val_root_dir, 'images')))
label_list_val = sorted(os.listdir(os.path.join(val_root_dir, 'annotations')))

image_list_test = sorted(os.listdir(os.path.join(test_root_dir, 'images')))
label_list_test = sorted(os.listdir(os.path.join(test_root_dir, 'annotations')))

In [ ]:
def image_label_pairs(image_list,label_list,root_dir):
  img_label_pair = {}
  for idx in range(len(image_list)):
    img_path = os.path.join(root_dir, 'images', image_list[idx])
    label_path = os.path.join(root_dir, 'annotations', label_list[idx])
    img = Image.open(img_path)
    img = img.convert('RGB')
    classes = {}
    with open(label_path, 'r') as f:
      for line in f:
        l2 = line.split(',')
        label = l2[-2]
        if l2[-2] not in classes:
          classes[l2[-2]] = 1
        else:
          classes[l2[-2]] += 1
    max_value = max(classes.values())
    max_keys = [k for k, v in classes.items() if v == max_value]
    label = max_keys[0]

    img_label_pair[idx] = []
    img_label_pair[idx].append(img)
    img_label_pair[idx].append(int(label))

  return img_label_pair

In [ ]:
img_label_pairs_train = image_label_pairs(image_list_train,label_list_train,train_root_dir)
img_label_pairs_val = image_label_pairs(image_list_val,label_list_val,val_root_dir)
img_label_pairs_test = image_label_pairs(image_list_test,label_list_test,test_root_dir)

In [ ]:
def reshape_img(image, target_shape):
    resized_image = cv2.resize(image, target_shape, interpolation=cv2.INTER_AREA)

    if resized_image.shape[0] < target_shape[0] or resized_image.shape[1] < target_shape[1]:
        pad_height = max(target_shape[0] - resized_image.shape[0], 0)
        pad_width = max(target_shape[1] - resized_image.shape[1], 0)
        resized_image = cv2.copyMakeBorder(
            resized_image, 0, pad_height, 0, pad_width,
            cv2.BORDER_CONSTANT, value=(0, 0, 0)
        )

    return resized_image

In [ ]:
def xdata_ydata_array(img_label_pairs):
  x_data = []
  y_data = []
  for idx in range(len(img_label_pairs)):
    image = np.array(img_label_pairs[idx][0])
    image = reshape_img(image, (224, 224))
    x_data.append(image)
    y_data.append(img_label_pairs[idx][1])

  x_data = np.array(x_data)
  y_data = np.array(y_data)
  y_data = tf.one_hot(y_data.astype(np.int32), depth=2)
  return x_data,y_data

In [ ]:
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

softmax = np.empty(outputSize)

In [ ]:
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
image = input_data[0]

In [ ]:
def calculate_softmax(data):
    result = np.exp(data)
    return result

In [ ]:
num_pics  = 10
fix, ax = plt.subplots(1, num_pics, figsize=(12,12))
plt.tight_layout()
for i in range(num_pics):
    image[0,...] = test_data[i]
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = calculate_softmax(temp[0][0])
    prediction = softmax.argmax()

    ax[i].set_title('Prediction: {}'.format(prediction))
    ax[i].axis('off')
    print(np.shape(test_data[i]))
    ax[i].imshow(test_data[i,:,:,4:].reshape(224,224,3), 'rgb')

In [ ]:
total = test_data.shape[0]
predictions = np.empty_like(test_label)
print("Classifying {} digit pictures ...".format(total))

start = time()
for i in range(total):
    image[0,...] = test_data[i]
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = calculate_softmax(temp[0][0])
    predictions[i] = softmax.argmax()

stop = time()
correct = np.sum(predictions==test_label)
execution_time = stop-start
print("Overall accuracy: {}".format(correct/total))
print("  Execution time: {:.4f}s".format(execution_time))
print("      Throughput: {:.4f}FPS".format(total/execution_time))

In [ ]:
del overlay
del dpu